In [5]:
colab = True

if colab:
    from google.colab import drive
    drive.mount('/content/gdrive')
    !git clone https://github.com/deepmind/pycolab.git
    !git clone https://github.com/nicoladainese96/RelationalModule.git
    !pip install pycolab

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
fatal: destination path 'pycolab' already exists and is not an empty directory.
fatal: destination path 'RelationalModule' already exists and is not an empty directory.


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from RelationalModule import ActorCritic, ControlActorCritic
from RelationalModule import train_agent as train
from RelationalModule import utils
from importlib import reload

In [2]:
reload(train)

<module 'RelationalModule.train_agent' from '/content/RelationalModule/train_agent.py'>

In [0]:
game_params = dict(grid_size=7,
                solution_length=[0],
                num_forward = [0], # number of distractors
                num_backward=[0], # just set to 0 for now
                branch_length=1, # length of forward distractors
                max_num_steps = 50
               )

In [6]:
HPs = dict(action_space=4,
           lr=0.003,
           gamma=0.99,
           TD=True,
           twin=True,
           tau=0.2,
           n_steps=40
           )
if colab:
    HPs['device'] = 'cuda'
else:
    HPs['device'] = 'cpu'

print('device: ', HPs['device'])  

# Relational Agent
agent = ActorCritic.BoxWorldA2C(**HPs)

# Control Agent
control_agent = ControlActorCritic.ControlA2C(**HPs)

device:  cuda


In [0]:
# Random Agent

class RandomAgent():
    def __init__(self, n_actions):
        self.n_actions = n_actions
    
    def get_action(self,state, *args, **kwargs):
        a = np.random.choice(self.n_actions)
        log_prob = np.log(1./self.n_actions) # just because it's the standard output of the other agent
        return a, log_prob
    
    def update(self, *args):
        return

rnd_agent = RandomAgent(4)

In [8]:
%%time
results = train.train_boxworld(agent, game_params, n_episodes=5000, 
                               max_steps=game_params['max_num_steps'], return_agent=True)
score, asymptotic_score, asymptotic_std, trained_agent, time_profile = results

/usr/local/lib/python3.6/dist-packages/pycolab/ascii_art.py:318: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  art = np.vstack(np.fromstring(line, dtype=np.uint8) for line in art)


Episode 100 - reward: 0.60
Episode 200 - reward: 0.60
Episode 300 - reward: 0.90
Episode 400 - reward: 0.40
Episode 500 - reward: 0.90
Episode 600 - reward: 1.00
Episode 700 - reward: 0.50
Episode 800 - reward: 0.80
Episode 900 - reward: 0.60


KeyboardInterrupt: ignored

In [0]:
plt.figure(figsize=(8,6))
n_epochs = np.arange(100, len(score))
average_score = np.array([np.mean(score[i:i+100]) for i in range(len(score)-100)])
plt.plot(n_epochs, average_score, alpha=0.9)
plt.title("Performance", fontsize=16)
plt.xlabel("Number of epochs", fontsize=16)
plt.ylabel("Total reward", fontsize=16)
plt.show()

In [0]:
save = True
keywords = ['relational', 'residual','unboxed_gem',str(len(control_score))+"-episodes","50-steps"] # example

if colab and save:
    %cd ~
    parent_dir = "/content/gdrive/My Drive/Colab Notebooks/"
    save_dir  = "RelationalTrained/"
    %cd "{parent_dir}"
    !mkdir "{save_dir}"
    ID = utils.save_session(save_dir, keywords, game_params, HPs, score)
    torch.save(trained_agent, save_dir+"agent_"+ID)